In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))
sys.path.append(os.path.join(os.getcwd(), "molgen"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
# from molgen.dataloading.MolGraphBatchPreprocessor import MolGraph
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model
from otgnn.graph import SYMBOLS, FORMAL_CHARGES, BOND_TYPES


from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, load_checkpoint
from mol_opt.mol_opt import MolOpt
from mol_opt.decoder_mol_opt import MolOptDecoder
from mol_opt.ot_utils import encode_target
from mol_opt.ot_utils import FGW 
from mol_opt.ot_utils import compute_barycenter

from molgen.dataloading.feat2smiles import feat2smiles
from molgen.dataloading.mol_drawer import MolDrawer
from molgen.metrics.Penalty import Penalty
from molgen.metrics.mol_metrics import MolMetrics

from rdkit.Chem import MolFromSmiles
from tensorboardX import SummaryWriter

import torch
import torch.nn.functional as F
import numpy as np
import time
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from molgen.metrics.Penalty import Penalty as PenaltyNew

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')

/home/octav/gitrepos/tum-thesis


In [2]:
model_type = "pointwise"
sys.argv = ["", "-cuda", "-model_type", model_type, "-one_batch_train"]
args = get_args()
args.init_model = f"{model_type}10-onebatch"
args.output_dir = "mol_opt/output_{}/".format(args.init_model)
args.tb_logs_dir = "mol_opt/logs_{}/".format(args.init_model)
args.init_model = "{}10-onebatch-euler-valency-scale2".format(model_type)
args.init_decoder_model = "{}_decode".format(args.init_model)
args.batch_size = 50
args.conn_penalty_function = 'logdet'
args

Namespace(N_transformer=6, agg_func='sum', annealing_rate=0.0001, batch_norm=False, batch_size=50, conn_penalty_function='logdet', connectivity=True, connectivity_hard=False, connectivity_lambda=0.025, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, euler_characteristic_penalty=True, euler_lambda=0.3, ffn_activation='LeakyReLU', init_decoder_model='pointwise10-onebatch-euler-valency-scale2_decode', init_model='pointwise10-onebatch-euler-valency-scale2', linear_out=False, model_type='pointwise', n_epochs=10, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=250, n_labels=1, n_layers=5, one_batch_train=True, ot_solver='emd', output_dir='mol_opt/output_pointwise10-onebatch/', pc_hidden=100, pred_hidden=150, scale_lambdas=False, sinkhorn_entropy=0.1, sinkhorn_max_it=10000, task='qed', tb_logs_dir='mol_opt/logs_pointwise10-onebatch/', valency=True, valency_hard=False, valency_lambda=0.07)

In [11]:
model = args.init_model
model_iter = 1000 

model_name = "model_{}_{}".format(model, model_iter)
print(args.output_dir + model_name)
model_decode_name = "model_{}_decode_{}".format(model, model_iter)

molopt, molopt_decoder, _, _, config, _ = load_checkpoint(args.output_dir + model_name)

metrics = MolMetrics(SYMBOLS, FORMAL_CHARGES, BOND_TYPES, False)
fgw_loss = FGW(alpha = 0.5)
pen = PenaltyNew(config, model_iter)

molopt_module_list = torch.nn.ModuleList([molopt, molopt_decoder])

# molopt = MolOpt(args)
# molopt_decoder = MolOptDecoder(args)
print (molopt)
print (molopt_decoder)
print(molopt.args.model_type)
pen.log()

mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_1000
MolOpt(
  (GCN): GCN(
    (W_message_i): Linear(in_features=100, out_features=250, bias=False)
    (W_message_h): Linear(in_features=250, out_features=250, bias=False)
    (W_message_o): Linear(in_features=343, out_features=100, bias=True)
    (W_mol_h): Linear(in_features=100, out_features=100, bias=True)
    (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
    (dropout_gcn): Dropout(p=0.0, inplace=False)
    (dropout_ffn): Dropout(p=0.0, inplace=False)
  )
  (opt0): Linear(in_features=100, out_features=250, bias=True)
  (opt1): Linear(in_features=250, out_features=100, bias=True)
)
MolOptDecoder(
  (fc1_SYMBOLS): Linear(in_features=100, out_features=150, bias=True)
  (fc2_SYMBOLS): Linear(in_features=150, out_features=64, bias=True)
  (fc1_CHARGES): Linear(in_features=100, out_features=150, bias=True)
  (fc2_CHARGES): Linear(in_features=150, out_features=5, bias=True)
  (fc1_BONDS): 

In [6]:
metrics.no_bond_type

4

In [7]:
[x is None for x in metrics.BOND_TYPES]

[False, False, False, False, True]

In [8]:
train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, True)

for i in train_data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

In [17]:
x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize_argmax(*yhat_logits)
yhat_labels = molopt_decoder.discretize(*yhat_logits, tau = pen.tau)
pred_pack = (yhat_labels, yhat_logits, Y.scope), Y 

target = Y.get_graph_outputs()
symbols_labels, charges_labels, bonds_labels = yhat_labels
symbols_logits, charges_logits, bonds_logits = yhat_logits

print (fgw_loss(*pred_pack, tau = 1).item()/args.batch_size)
pen_loss = pen(*pred_pack, model_iter)
print (pen_loss)
metrics.measure_batch(pred_pack[0], target)

0.8033094787597657
(tensor(0., device='cuda:0'), tensor(16.8787, device='cuda:0', grad_fn=<AddBackward0>), tensor(4.1408, device='cuda:0', grad_fn=<AddBackward0>))


{'molecular_disconnected_validity': 4.0,
 'molecular_validity': 0.0,
 'symbol_accuracy': 76.36700648748841,
 'connected_components': 3.64,
 'invalid_valency_nodes': 20.20389249304912,
 'avg_euler_error': 0.69,
 'invalid_euler_toofew': 36.0,
 'invalid_euler_toomany': 14.000000000000002}

In [14]:
metrics.batch_euler_num_edges(pred_pack[0], target)

{'avg_euler_error': 1.05,
 'invalid_euler_toofew': 34.0,
 'invalid_euler_toomany': 20.0}

In [15]:
torch.sum(~(bonds == 4))//2

NameError: name 'bonds' is not defined

In [16]:
bond_idx = 0
smiles = []
for mol_idx, (st, le) in enumerate(Y.scope):
    pred_symbols = symbols_labels[st:st+le]
    target_symbols = torch.argmax(target[mol_idx]['SYMBOLS'], dim=1)
    
    symbols = symbols_labels[st:st+le]
    charges = charges_labels[st:st+le]
    bonds = bonds_labels[bond_idx:bond_idx+le*le].view(le, le, -1)
    bond_idx += le * le
    smiles.append(feat2smiles(SYMBOLS, FORMAL_CHARGES, BOND_TYPES, symbols, charges, bonds, False))
    print (smiles[-1])

C.C.C.C.C[AsH]CO.C[Sb]1CCC1[BiH]CSCC1CCC1.[SbH3]
C.C.C.C.C.C.C.Cc1oC2O(C)C[Sb]12.N.[AsH3].[SeH2]
C.C.C.CC12C(cn)[Sb]3(C)(C)C4NC1c41([sbH])(C[Te][SbH]1)C23.S
C.C.C.C.C.C.C#S(C)(S)CNC.CC.CC.N
C.C.CC12(C)CC([AlH]C(O)N1)C2[SbH2].C[AsH2].[SiH4].c[se]1CNC1
C.CC1Cc23(Cs45c6[te]7C4[SiH]4CO64(C)C75)c[AsH]C2NC13
CC1C2C3C456C7C8C9C2%10(C)C(C)SC1%10[sb]9312%10(C)c84(O)c73c5(C(=C31)C2)C6%10
C.C.C.C.C.CC.CC[AsH2].CN1C2(P)C3C[SiH2]C31[As]2C
C.C.C[BiH]C.cc1C(N)[Sb](C)C1N1(C[Te]C)C[PH](C)(C[BiH2])C1
C.C.C.C.C.CN[Sb](C)SC.Cco(C)C.N[SbH][SeH]
Co1sC2CCC23(N)c2n456cC27(C)(C)CN(C4[Sb]3C5)[As]1C67
C.C.cs(C1S23C4(C)CC256[SbH]C3(C)([SiH]5CN)C146(C)C)N12C[BiH]C1[SeH]2C
C.C.C.C.C.C.C.C.C.C.O.P.ccC.ccC1=[As](C)(C)CC1.cs
C.C.C.C.C[Sb]=[PH]1(N)=N(C)C([SbH]23C4cc(c4)(CC2[SbH2])C3[TeH])N1
C.CC12[Te][Al]3CC3CC13=[SH]C3O1C23[As]2C45CC267#[BiH]42C(N56)[TeH]3S12C7
cc(CCC)c1([BiH2])-[sb](c)([SbH2])cS1(C)(C)(C)(C)([AsH2])C[SiH3]
C.C.CNCN1CC23co(C2)C13[BiH2].CS.ccCCC
C.C.CCC1(N)(N)(CC)SC[As]2C[AsH]13[BiH]C1(P)CC(C)n1c23[SbH